# Análisis Financiero del Russell 1000 usando YahooQuery

## Etapa 1: Obtener listado del Russell 1000

In [ ]:
import pandas as pd
import requests
import os
import numpy as np
# URL de la página
url = 'https://en.wikipedia.org/wiki/Russell_1000_Index'

# Encabezado para evitar el error 403
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)'
}

# Hacer la solicitud
response = requests.get(url, headers=headers)

# Leer las tablas HTML desde el contenido descargado
tables = pd.read_html(response.text)

# Inspeccionar la tabla 3 (asegúrate de que contenga los tickers)
df_r1000 = tables[3]  # O cambia a otro índice si no es la tabla correcta

# Mostrar las columnas disponibles
print(df_r1000.columns)

# Extraer los tickers si existe la columna 'Symbol'
if 'Symbol' in df_r1000.columns:
    tickers = df_r1000['Symbol'].tolist()
    tickers=tickers[:300]
else:
    print("❌ La columna 'Symbol' no está en esta tabla.")


Index(['Company', 'Symbol', 'GICS Sector', 'GICS Sub-Industry'], dtype='object')


C:\Users\juans\AppData\Local\Temp\ipykernel_25328\1362923871.py:17: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(response.text)


## Etapa 2: Descargar EPS básico (últimos 4 años fiscales)

In [4]:
from yahooquery import Ticker

# Obtener datos financieros para todos los tickers de una vez
t = Ticker(tickers)
types=['BasicEPS','CashCashEquivalentsAndShortTermInvestments', 'EBITDA', 'TotalRevenue', 'InterestExpense', 'TaxProvision', 'ChangeInWorkingCapital','LongTermDebtAndCapitalLeaseObligation','FreeCashFlow','BasicAverageShares']
eps_data = t.get_financial_data(types,frequency='a', trailing=False)
data_quarter=t.get_financial_data(types,frequency='q', trailing=False)

## Etapa 3: Filtrar empresas con 4 años de EPS positivo

In [5]:
# Filtrar empresas con valores negativos en variables críticas
eps_data = eps_data[
    (eps_data['EBITDA'] > 0) &
    (eps_data['TotalRevenue'] > 0)
]
# Reemplazar InterestExpense nulos por 0 (caso típico)
eps_data['InterestExpense'] = eps_data['InterestExpense'].fillna(0)
eps_data['CashCashEquivalentsAndShortTermInvestments'] = eps_data['CashCashEquivalentsAndShortTermInvestments'].fillna(0)
eps_data['TaxProvision'] = eps_data['TaxProvision'].fillna(0)
eps_data['LongTermDebtAndCapitalLeaseObligation'] = eps_data['LongTermDebtAndCapitalLeaseObligation'].fillna(0)

#Margin Ebitda
eps_data['EBITDA_Margin'] = eps_data['EBITDA'] / eps_data['TotalRevenue']
#Límite 1
Den = eps_data['EBITDA'] - eps_data['TaxProvision'] - eps_data['ChangeInWorkingCapital']
eps_data['Límite1'] = eps_data['InterestExpense'] / Den

#Límite 2
num = eps_data['LongTermDebtAndCapitalLeaseObligation'] - eps_data['CashCashEquivalentsAndShortTermInvestments']
Den2 = eps_data['EBITDA']
eps_data['Límite2'] = num / Den

# Mantener solo tickers con al menos 4 registros de EPS
ticker_counts = eps_data.index.get_level_values(0).value_counts()
tickers_validos = ticker_counts[ticker_counts >= 4].index.tolist()

# Filtrar dataset para dejar solo esas empresas
eps_filtrado = eps_data.loc[tickers_validos]


## Etapa 4: Reorganizar el EPS en un DataFrame limpio

In [6]:
# Resetear índice y limpiar columnas
eps_filtrado = eps_filtrado.reset_index()
eps_filtrado = eps_filtrado[['symbol', 'asOfDate', 'BasicEPS', 'EBITDA_Margin', 'Límite1','Límite2']]
eps_filtrado['asOfDate'] = pd.to_datetime(eps_filtrado['asOfDate'])

# Agregar filas con +3 días para cubrir posibles feriados/no cotización
import datetime

rows_extra = []
for idx, row in eps_filtrado.iterrows():
    new_date = row['asOfDate'] + datetime.timedelta(days=3)
    rows_extra.append({'symbol': row['symbol'], 'asOfDate': new_date, 'BasicEPS': row['BasicEPS'],'EBITDA_Margin': row['EBITDA_Margin'],'Límite1': row['Límite1'],'Límite2': row['Límite2']})

eps_fiscal = pd.concat([eps_filtrado, pd.DataFrame(rows_extra)], ignore_index=True)


## Etapa 5: Descargar precios históricos (últimos 5 años)

In [8]:

from datetime import datetime, timedelta
import os


START_DATE = "2020-05-30"          # Fecha inicial fija para descargar históricos
WEEKS_BACK = 2                     # Cuántas semanas antes de hoy descargar
PARQUET_FILE = "prices.parquet"    # Archivo local donde se guardan los datos


# CALCULAR FECHA FINAL (end_date)
# Dos semanas antes de la fecha actual
# .normalize() asegura que no tenga hora, para evitar problemas al comparar días

today = pd.Timestamp.today().normalize()
end_date = today - timedelta(days=WEEKS_BACK * 7)

# Ajustar end_date al último día hábil (lunes-viernes)
while end_date.weekday() > 4:
    end_date -= timedelta(days=1)

# SI EL ARCHIVO YA EXISTE, CARGARLO Y VERIFICAR SI NECESITA ACTUALIZACIÓN

if os.path.exists(PARQUET_FILE):

    # Cargar datos existentes
    prices = pd.read_parquet(PARQUET_FILE)

    # Última fecha disponible en el archivo local (sin hora)
    last_date = prices["asOfDate"].max().normalize()

    # Si el archivo ya está actualizado, no se descarga nada
    if last_date >= end_date:
        print(f"Datos actualizados hasta {last_date.date()}, no se descarga nada.")

    else:
        
        print(f"Actualizando datos desde {last_date.date()} hasta {end_date.date()}...")

        # Lista de tickers válidos ya almacenados
        tickers_validos = prices["symbol"].unique().tolist()

        # Crear objeto Ticker
        t = Ticker(tickers_validos)

        # --- Ajustar update_start al siguiente día hábil ---
        update_start_date = last_date + timedelta(days=1)
        while update_start_date.weekday() > 4:
            update_start_date += timedelta(days=1)

        # Si update_start es posterior a end_date, no hay nada que descargar
        if update_start_date > end_date:            
            print("No hay días hábiles nuevos para descargar (update_start > end_date).")
        else:
            start_str = update_start_date.strftime("%Y-%m-%d")
            end_str = end_date.strftime("%Y-%m-%d")

            # Descargar solo la parte faltante (fechas ya ajustadas a días hábiles)
            new_data = t.history(start=start_str, end=end_str, interval="1d")

            # Si yahoo devuelve None o un DataFrame vacío, saltar la concatenación
            if new_data is None or (hasattr(new_data, "empty") and new_data.empty):
                print("⚠️ Yahoo no devolvió nuevos precios para el rango solicitado.")
            else:
                new_data = (
                    new_data
                    .drop(columns=['open', 'high', 'low', 'volume', 'adjclose', 'dividends', 'splits'], errors='ignore')
                    .reset_index()[['symbol', 'date', 'close']]
                    .rename(columns={'date': 'asOfDate'})
                )
                new_data['asOfDate'] = pd.to_datetime(new_data['asOfDate']).dt.normalize()

                # Concatenar y eliminar duplicados
                prices = pd.concat([prices, new_data], ignore_index=True)
                prices = prices.drop_duplicates(subset=['symbol', 'asOfDate'])

                prices.to_parquet(PARQUET_FILE, index=False)
                print("💾 prices.parquet actualizado")

else:
  
    # PRIMERA DESCARGA (si no existe el archivo)

    print("Descargando datos históricos por primera vez...")

    # t_valid ya debería existir antes de esta etapa con los tickers válidos finales
    t = Ticker(tickers_validos)

    # Descargar histórico completo una sola vez
    prices = t.history(start=START_DATE, end=end_date, interval="1d")

    # Limpiar columnas
    prices = (
        prices
        .drop(columns=['open', 'high', 'low', 'volume', 'adjclose', 'dividends', 'splits'])
        .reset_index()[['symbol', 'date', 'close']]
        .rename(columns={'date': 'asOfDate'})
    )

    # Normalizar fecha (sin hora)
    prices['asOfDate'] = pd.to_datetime(prices['asOfDate']).dt.normalize()

    # Guardar archivo
    prices.to_parquet(PARQUET_FILE, index=False)

print("Proceso completado.")



Descargando datos históricos por primera vez...


c:\Users\juans\AppData\Local\Programs\Python\Python312\Lib\site-packages\yahooquery\utils\__init__.py:1470: FutureWarning: 'S' is deprecated and will be removed in a future version. Please use 's' instead of 'S'.
  has_live_indice = index_utc[-1] >= last_trade - pd.Timedelta(2, "S")
c:\Users\juans\AppData\Local\Programs\Python\Python312\Lib\site-packages\yahooquery\utils\__init__.py:1470: FutureWarning: 'S' is deprecated and will be removed in a future version. Please use 's' instead of 'S'.
  has_live_indice = index_utc[-1] >= last_trade - pd.Timedelta(2, "S")
c:\Users\juans\AppData\Local\Programs\Python\Python312\Lib\site-packages\yahooquery\utils\__init__.py:1470: FutureWarning: 'S' is deprecated and will be removed in a future version. Please use 's' instead of 'S'.
  has_live_indice = index_utc[-1] >= last_trade - pd.Timedelta(2, "S")
c:\Users\juans\AppData\Local\Programs\Python\Python312\Lib\site-packages\yahooquery\utils\__init__.py:1470: FutureWarning: 'S' is deprecated and wil

Proceso completado.


c:\Users\juans\AppData\Local\Programs\Python\Python312\Lib\site-packages\yahooquery\utils\__init__.py:1470: FutureWarning: 'S' is deprecated and will be removed in a future version. Please use 's' instead of 'S'.
  has_live_indice = index_utc[-1] >= last_trade - pd.Timedelta(2, "S")
c:\Users\juans\AppData\Local\Programs\Python\Python312\Lib\site-packages\yahooquery\utils\__init__.py:1470: FutureWarning: 'S' is deprecated and will be removed in a future version. Please use 's' instead of 'S'.
  has_live_indice = index_utc[-1] >= last_trade - pd.Timedelta(2, "S")
c:\Users\juans\AppData\Local\Programs\Python\Python312\Lib\site-packages\yahooquery\utils\__init__.py:1470: FutureWarning: 'S' is deprecated and will be removed in a future version. Please use 's' instead of 'S'.
  has_live_indice = index_utc[-1] >= last_trade - pd.Timedelta(2, "S")
c:\Users\juans\AppData\Local\Programs\Python\Python312\Lib\site-packages\yahooquery\utils\__init__.py:1470: FutureWarning: 'S' is deprecated and wil

## Etapa 6: Unir EPS con precios para calcular PER

In [9]:
# Merge con símbolo y fecha
df_merge = pd.merge(prices, eps_fiscal, on=['symbol', 'asOfDate'])

# Filtrar empresas con 4 años válidos
valid_counts = df_merge['symbol'].value_counts()
df_merge = df_merge[df_merge['symbol'].isin(valid_counts[valid_counts >= 4].index)]

# Calcular PER
df_merge['PER'] = df_merge['close'] / df_merge['BasicEPS']

# Valoración por múltiplos
Resumen = df_merge.sort_values(['symbol', 'asOfDate'])

promedios = (
    Resumen.groupby('symbol')
    .agg(PER_promedio=('PER', 'mean'),
         EPS_promedio=('BasicEPS', 'mean'))
    .reset_index()
)

# Calcular precio objetivo con PER promedio y EPS promedio
promedios['Precio_Objetivo'] = promedios['EPS_promedio'] * promedios['PER_promedio']
promedios['Precio_Objetivo_85'] = promedios['Precio_Objetivo'] * 0.85

# Filtrar empresas con PER promedio < 40
promedios_filtrados = promedios[promedios['PER_promedio'] < 40]

# Obtener lista de símbolos válidos
tickers_filtrados = promedios_filtrados['symbol'].tolist()


## Etapa 7: VALORACIÓN FLUJO DE CAJA

In [ ]:
# ================================
# ETAPA 7: CALCULAR VALOR JUSTO (DCF)
# ================================

print("\n🔍 Iniciando Etapa 7 — Valoración DCF...\n")

# Función para calcular valor justo
def calculate_stock_value(ticker, annual_data, quarterly_data):

    quarterly_data = quarterly_data.tail(4).fillna(method='ffill')

    if quarterly_data.empty or annual_data.empty:
        return np.nan

    # Ingresos y FCF
    revenues_quarterly = quarterly_data.get('TotalRevenue', pd.Series([0]*len(quarterly_data)))
    cash_flow_quarterly = quarterly_data.get('FreeCashFlow', pd.Series([0]*len(quarterly_data)))
    cash_quarterly = quarterly_data.get('CashCashEquivalentsAndShortTermInvestments',
                                        pd.Series([0]*len(quarterly_data)))

    debt_quarterly = quarterly_data.get('LongTermDebtAndCapitalLeaseObligation',
                                        pd.Series([0]*len(quarterly_data)))
    
    shares_quarterly = quarterly_data.get('BasicAverageShares',
                                          pd.Series([1]*len(quarterly_data)))

    # Sumar últimos 4 trimestres
    revenue_0 = revenues_quarterly.sum()
    cash_flow_0 = cash_flow_quarterly.sum()
    cash_0 = cash_quarterly.iloc[-1]
    debt_0 = debt_quarterly.iloc[-1]
    shares_0 = shares_quarterly.iloc[-1]

    if shares_0 == 0:
        return np.nan

    discount_rate = 0.09

    # Crecimiento histórico (anual)
    revenues_annual = annual_data.get('TotalRevenue', pd.Series([0]*len(annual_data)))
    cash_flow_annual = annual_data.get('FreeCashFlow', pd.Series([0]*len(annual_data)))

    revenue_growth = revenues_annual.pct_change().dropna()
    fcf_margin = (cash_flow_annual / revenues_annual).dropna()

    if revenue_growth.empty or fcf_margin.empty:
        return np.nan

    growth_rates = {
        'pesimista': revenue_growth.mean()*0.2,
        'moderado': revenue_growth.mean(),
        'optimista': revenue_growth.max()
    }

    # Proyección de FCF 5 años
    fcf_projections = {
        scenario: [cash_flow_0 * (1 + growth_rate)**year for year in range(6)]
        for scenario, growth_rate in growth_rates.items()
    }

    # Terminal Value
    perpetual_growth = {'pesimista':0.01,'moderado':0.015,'optimista':0.02}

    perpetual_fcf = {
        s: fcf_projections[s][-1] * (1 + perpetual_growth[s]) / (discount_rate - perpetual_growth[s])
        for s in growth_rates
    }

    # VPN
    vpn = {
        s: np.sum([fcf/((1+discount_rate)**(i+1)) for i, fcf in enumerate(fcf_projections[s])]) +
           perpetual_fcf[s] / ((1+discount_rate)**6)
        for s in growth_rates
    }                                                                
    fair_value = np.mean([(vpn[s] + cash_0 - debt_0) / shares_0 for s in growth_rates])

    return fair_value


# ================================
# Ejecutar valoración sobre tickers válidos
# ================================

stock_values = {}
for ticker in tickers_validos:
    try:
        print(f"📈 Calculando valor para: {ticker}")

        # ANUAL = eps_data
        annual_data = eps_data.loc[ticker].reset_index().sort_values("asOfDate")

        # TRIMESTRAL = data_quarter 
        quarterly_data = data_quarter.loc[ticker].reset_index().sort_values("asOfDate")

        stock_values[ticker] = calculate_stock_value(ticker, annual_data, quarterly_data)

    except Exception as e:
        print(f"Error procesando {ticker}: {e}")
        stock_values[ticker] = np.nan

print("\n🏁 ETAPA 7 COMPLETADA\n")



🔍 Iniciando Etapa 7 — Valoración DCF...

📈 Calculando valor para: CNXC
{'pesimista': 0.04059451824106956, 'moderado': 0.2029725912053478, 'optimista': 0.3519743472182828}

🏁 ETAPA 7 COMPLETADA



C:\Users\juans\AppData\Local\Temp\ipykernel_25328\2835942000.py:10: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  quarterly_data = quarterly_data.tail(4).fillna(method='ffill')


## Etapa 8: Obtener precios actuales

In [ ]:
from datetime import date
t_validf = Ticker(tickers_filtrados)
precios_actuales = t_validf.history(start='2025-09-24', end='2025-09-25', interval='1d')
precios_actuales = precios_actuales.reset_index()[['symbol', 'close']]
precios_actuales = precios_actuales.groupby('symbol').last().reset_index()
precios_actuales.rename(columns={'close': 'Precio_Actual'}, inplace=True)


## Etapa 9: Resultado final y exportación

In [ ]:
df_final = pd.merge(promedios_filtrados, precios_actuales, on='symbol', how='inner')

# Reordenar columnas y guardar
df_final = df_final[['symbol', 'Precio_Actual', 'Precio_Objetivo_85', 'PER_promedio', 'EPS_promedio']]
df_final.columns = ['Empresa', 'Precio Actual', 'Precio Objetivo', 'PER Promedio', 'EPS Promedio']

# Exportar
df_final.to_excel('Russell_1000_Valoraciones.xlsx', index=False)
print("Archivo guardado como 'Russell_1000_Valoraciones.xlsx'")


Archivo guardado como 'Russell_1000_Valoraciones.xlsx'


# Etapa 10 – Evolución anual del Margen EBITDA por empresa

In [ ]:


# Filtrar eps_fiscal con los tickers que tienen PER promedio < 40
eps_filtrado_final = eps_fiscal[eps_fiscal['symbol'].isin(tickers_filtrados)].copy()

# Agregar columna de año
eps_filtrado_final['Año'] = eps_filtrado_final['asOfDate'].dt.year

# Calcular promedios anuales
margen_anual = eps_filtrado_final.groupby(['symbol', 'Año'])[['EBITDA_Margin', 'Límite1']].mean().reset_index()
margen_anual.rename(columns={
    'symbol': 'Empresa',
    'EBITDA_Margin': 'Margen EBITDA',
    'Límite1': 'Límite1'
}, inplace=True)

# Exportar a nueva hoja en el Excel existente
from openpyxl import load_workbook
with pd.ExcelWriter('Russell_1000_Valoraciones.xlsx', mode='a', engine='openpyxl') as writer:
    margen_anual.to_excel(writer, sheet_name='Margen EBITDA por Año', index=False)

print("Guardado 'Margen EBITDA por Año' con empresas que tienen PER Promedio < 40.")


Guardado 'Margen EBITDA por Año' con empresas que tienen PER Promedio < 40.
